In [2]:
!pip install torch

In [1]:
#from transformers import pipeline
from optparse import OptionParser
#from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import json

In [3]:
json_path="/home/shawn/Downloads/01_test.json"

with open(json_path) as json_file:
    schema_dictionary = json.load(json_file)

In [4]:
def schemaToDictionary(schemaList):
    '''
        Takes nodes or edges list and makes a dictionary you can index list elements using the ID, probably
        only worth running this function if you have a really big graph you need to access nodes or edges of
        many times, but this option is available for that
    '''
    newDict={}
    for dictionary in schemaList:
        newDict[dictionary['id']] = dictionary
    return newDict

In [5]:
def findRoots(schema_dictionary):
    '''
        Finds the roots of a schema
        1. Get all the ids of all nodes, put in a stack
        2. Check what targets are currently in the schema
        3. Remove them as you find them

        Give schema dictionary (direct from json file), a dictionary
        Returns the stack of roots of the tree, False if schema nodes have no sources
    '''
    stack = []
    for node in schema_dictionary['nodes']:
        stack.append(node['id'])
    for edge in schema_dictionary['edges']:
        if edge['target'] in stack:
            stack.remove(edge['target'])
    return stack

In [6]:
def removeNodeIDs(id_list, schema_dictionary):
    '''
        Takes a list of id's to be removed and returns a new schema dictionary with the node
        id's removed

        Note, when making graph smaller, it will usually make more sense to remove unique
        edge id's than nodes. However if there is some reason to this node removal function is here

        This is also really slow, consider changing if u actually make graphs really big
    '''
    new_dictionary = schema_dictionary
    for id in id_list:
        for node in new_dictionary['nodes']:
            if node['id'] == id:
                new_dictionary['nodes'].remove(node)
    return new_dictionary

In [7]:
def removeEdgeIDs(id_list, schema_dictionary):
    '''
        takes a list of edge id's and returns a new schema dictionary with the edge id's removed
        Note: important that you remove by edgeid, not source or target since the edge id is unique
    '''
    new_dictionary = schema_dictionary
    for id in id_list:
        for edge in new_dictionary['edges']:
            if edge['id'] == id:
                new_dictionary['edges'].remove(edge)
    return new_dictionary

In [8]:
removeNodeIDs(findRoots(schema_dictionary), schema_dictionary)

TypeError: object of type 'bool' has no len()

In [9]:
schema_dictionary['edges']
#im pretty sure this flow was not made properly, to do the desired effect correctly can circumvent problem by closing loop from bottom to
#top, but this might be very confusing for users who will expect source and target to not be dependent on which side but on
#which node they drew the edge from, our first version will be side dependent but future versions should figure out on 
#client side where the edge started from and is going to source and target edges properly
#there should also be some visual indication of where inputs and outputs are going, again just publish the website quickly and fix this
#in an update

[{'id': 'e1-2', 'source': '1', 'target': '2'},
 {'source': '00001',
  'sourceHandle': 'bottom',
  'target': '3159559',
  'targetHandle': 'top',
  'id': 'reactflow__edge-00001bottom-3159559top'},
 {'source': '00001',
  'sourceHandle': 'right',
  'target': '3166128',
  'targetHandle': 'left',
  'id': 'reactflow__edge-00001right-3166128left'},
 {'source': '3166128',
  'sourceHandle': 'bottom',
  'target': '3171323',
  'targetHandle': 'top',
  'id': 'reactflow__edge-3166128bottom-3171323top'},
 {'source': '3159559',
  'sourceHandle': 'right',
  'target': '3171323',
  'targetHandle': 'left',
  'id': 'reactflow__edge-3159559right-3171323left'}]

In [10]:
test = schemaToDictionary(schema_dictionary['nodes'])

In [11]:
test['00001']

{'id': '00001',
 'type': 'textInput',
 'data': {'label': 'You are Frodo. Say something interesting, and respond to text above if you received any.'},
 'position': {'x': 195.185336410498, 'y': 46.87644239300134},
 'width': 289,
 'height': 50,
 'selected': False,
 'positionAbsolute': {'x': 195.185336410498, 'y': 46.87644239300134},
 'dragging': False}

In [17]:
schema_dictionary['nodes']

[{'id': '00001',
  'type': 'textInput',
  'data': {'label': 'You are Frodo. Say something interesting, and respond to text above if you received any.'},
  'position': {'x': 195.185336410498, 'y': 46.87644239300134},
  'width': 289,
  'height': 50,
  'selected': False,
  'positionAbsolute': {'x': 195.185336410498, 'y': 46.87644239300134},
  'dragging': False},
 {'id': '3159559',
  'type': 'textInput',
  'data': {'label': 'You are Sam. Say something interesting, and respond to text above if you received any.'},
  'position': {'x': 222, 'y': 292},
  'width': 324,
  'height': 50,
  'selected': False,
  'positionAbsolute': {'x': 222, 'y': 292},
  'dragging': False},
 {'id': '3166128',
  'type': 'textInput',
  'data': {'label': 'You are Legolas. Say something interesting, and respond to text above if you received any.'},
  'position': {'x': 570, 'y': 64.1},
  'width': 337,
  'height': 50,
  'selected': False,
  'positionAbsolute': {'x': 570, 'y': 64.1},
  'dragging': False},
 {'id': '3171323

In [ ]:
#to start with, I think we should always start from the first node created. May make sense in the future to allow user to specify
#what node they want to start with, but i am too lazy to think of what the UX of that would be 

In [13]:
def runLLM(node_id, schema_dictionary):
    '''
        put a funciton that runs the LLM here
    '''
    for node in schema_dictionary['nodes']:
        print(node)
        if node['id'] == node_id:
            prompt = ''
            for key in node['data'].keys():
                print(len(node['data'].keys()))
                prompt += node['data'][key]
                prompt += " \n"
            # run LLM on prompt, note that this output will need to be sent over web somehow
    #for testing just return the prompt
    return prompt

In [14]:
def runSchema(schema_dictionary):
    '''
        Take a schema and run the flow. 
        
        Need a function that finds the root of the tree first.

        Also needs a function to c

        There are 2 things to track, what the current graph looks like and the stack of 
        nodes to run and their received prompts. Nodes on the stack have to check if they are a 
        root, or else they are not run yet. Root nodes on the stack are run in order, then
        targets are added to the stack with their associated prompt. If the target is already
        on the stack, simply give it the additional prompt.

        If there are nodes, and all of the nodes have a source, this means that there is a loop 
        somewhere in the graph. We still want to run the flow, because recursive nodes are a 
        selling point. In this case, if ALL the nodes have a source, pick the most recently 
        created node and run it, sending its prompts and adding the new targets to the stack. We
        do not update the graph now, instead simply running the node, then its targets in order.
        This will continue running iterations, until a stop button is pressed.

        The user will probably want to

        We should create a stack, a list of dictionaries that look like this:
        [{node_id: ____, sources_dict:{source_id:received_prompt = "" }}]
        This forms a stack. Sources dict may or may not have multiple source_ids in it. Queue up
        the next nodes to run based on what the targets of the node you just ran are. If
        a node has multiple sources when it's run, combine those sources into one prompt
        via concatenation. Received prompts are added first, with the LLMs actual text
        entered into it added last. We may have to do prompt engineering to make sure
        the LLM answers the prompt inputted into the box and not questions received as context,
        but this is not preferred.
        For bonus points, add an option to use dfs or bfs
    '''
    seen_edges=[]
    seen_nodes=[]
    #entries of a stack [{'node_id':____, 'edge_id':____, prompts:[]}]
    next_stack=[]
    current_stack=[]

    roots = findRoots(schema_dictionary)
    print(roots)
    #if there are roots to schema, get all the edge ids and targets from the schema which that 
    #root is the source for. Run the roots. As you run the roots, put their outputs as a new entry
    #in data
    for root in roots:
        for edge in schema_dictionary['edges']:
            if edge['source']==root:
                edge_id = edge['id']
                runLLM(root, schema_dictionary)

In [15]:
runSchema(schema_dictionary)

TypeError: object of type 'bool' has no len()

In [39]:
schema_dictionary

{'nodes': [{'id': '00001',
   'type': 'textInput',
   'data': {'label': 'You are Frodo. Say something interesting, and respond to text above if you received any.'},
   'position': {'x': 195.185336410498, 'y': 46.87644239300134},
   'width': 289,
   'height': 50,
   'selected': False,
   'positionAbsolute': {'x': 195.185336410498, 'y': 46.87644239300134},
   'dragging': False},
  {'id': '3159559',
   'type': 'textInput',
   'data': {'label': 'You are Sam. Say something interesting, and respond to text above if you received any.'},
   'position': {'x': 222, 'y': 292},
   'width': 324,
   'height': 50,
   'selected': False,
   'positionAbsolute': {'x': 222, 'y': 292},
   'dragging': False},
  {'id': '3166128',
   'type': 'textInput',
   'data': {'label': 'You are Legolas. Say something interesting, and respond to text above if you received any.'},
   'position': {'x': 570, 'y': 64.1},
   'width': 337,
   'height': 50,
   'selected': False,
   'positionAbsolute': {'x': 570, 'y': 64.1},
   